In [1]:
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset
import tensorflow as tf
from keras.metrics import Precision, Recall
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (
    LSTM,
    Dense,
    Embedding,
    Bidirectional,
    Dropout,
    GlobalMaxPooling1D,
    Concatenate,
    BatchNormalization,
    Conv1D,
    ReLU,
    Input,
    GRU,
)
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras import regularizers
from keras.layers import Embedding, Flatten, Dense
import tensorflow_datasets as tfds
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk import word_tokenize, pos_tag, ne_chunk
import os
from keras.preprocessing.sequence import TimeseriesGenerator
from datetime import datetime
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import pipeline
from transformers import BertTokenizer, TFBertModel


2024-02-11 18:59:54.007461: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-11 18:59:54.007513: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-11 18:59:54.008814: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-11 18:59:54.015748: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-11 18:59:55.168552: W tensorflow/compiler/tf2

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased')

In [ ]:
input_text = "I feel utterly hopeless and alone in this bleak and desolate world"
input_ids = tokenizer.encode(input_text, add_special_tokens=True, max_length=128, truncation=True, padding='max_length', return_tensors="tf")
outputs = model(input_ids)
predicted_probs = tf.nn.softmax(outputs.logits)

print(predicted_probs)

In [5]:
unmasker = pipeline('fill-mask', model='bert-base-uncased')
unmasker("apple is an [MASK] in artificial intelligence.")

config.json: 100%|██████████| 570/570 [00:00<00:00, 2.87MB/s]
/workspaces/ml-learning/venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:992: UserWarning: Not enough free disk space to download the file. The expected file size is: 440.45 MB. The target location /home/codespace/.cache/huggingface/hub only has 139.49 MB free disk space.
  warnings.warn(
/workspaces/ml-learning/venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:992: UserWarning: Not enough free disk space to download the file. The expected file size is: 440.45 MB. The target location /home/codespace/.cache/huggingface/hub/models--bert-base-uncased/blobs only has 139.49 MB free disk space.
  warnings.warn(
model.safetensors: 100%|██████████| 440M/440M [00:02<00:00, 217MB/s] 


ValueError: Could not load model bert-base-uncased with any of the following classes: (<class 'transformers.models.auto.modeling_auto.AutoModelForMaskedLM'>, <class 'transformers.models.auto.modeling_tf_auto.TFAutoModelForMaskedLM'>, <class 'transformers.models.bert.modeling_bert.BertForMaskedLM'>, <class 'transformers.models.bert.modeling_tf_bert.TFBertForMaskedLM'>). See the original errors:

while loading with AutoModelForMaskedLM, an error is thrown:
Traceback (most recent call last):
  File "/workspaces/ml-learning/venv/lib/python3.10/site-packages/transformers/pipelines/base.py", line 278, in infer_framework_load_model
    model = model_class.from_pretrained(model, **kwargs)
  File "/workspaces/ml-learning/venv/lib/python3.10/site-packages/transformers/models/auto/auto_factory.py", line 566, in from_pretrained
    return model_class.from_pretrained(
  File "/workspaces/ml-learning/venv/lib/python3.10/site-packages/transformers/modeling_utils.py", line 3383, in from_pretrained
   

In [3]:

# Load pre-trained BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-medium-uncased')

# Load pre-trained BERT model
model = TFBertModel.from_pretrained('bert-medium-uncased')




OSError: bert-medium-uncased is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [ ]:
def cosine_similarity(embedding1, embedding2):
    dot_product = np.dot(embedding1, embedding2.T)
    norm1 = np.linalg.norm(embedding1)
    norm2 = np.linalg.norm(embedding2)
    similarity = dot_product / (norm1 * norm2)
    return similarity

In [ ]:
word1 = "angry"
word2 = "furious"

# Tokenize words
word1_tokens = tokenizer.encode(word1, add_special_tokens=False, return_tensors="tf")
word2_tokens = tokenizer.encode(word2, add_special_tokens=False, return_tensors="tf")

# Get BERT embeddings
word1_embedding = model(word1_tokens)[0][:, 0, :].numpy()  # Extract embeddings for [CLS] token
word2_embedding = model(word2_tokens)[0][:, 0, :].numpy()

similarity_score = cosine_similarity(word1_embedding, word2_embedding)
print(similarity_score)